<a href="https://colab.research.google.com/github/IM07813/Transformers/blob/main/essay_eval__.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Import libraries
import tensorflow as tf
import transformers
from transformers import TFAutoModelForSequenceClassification, AutoTokenizer

# Load pre-trained model and tokenizer
model_name = "bert-base-cased"
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6) # Assuming 6-point scale for essay scores
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(transformers.__version__)
print(tf.__version__)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


4.29.2
2.12.0


In [ ]:
# Import the necessary libraries
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization

# Load the pre-trained BERT model and tokenizer
model_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'
model = hub.KerasLayer(model_url, trainable=True)
tokenizer = hub.KerasLayer(model_url + '/preprocess')

# Define a function to preprocess the image and extract features using a CNN
def image_preprocess(image):
  # Resize and normalize the image
  image = tf.image.resize(image, (224, 224))
  image = tf.cast(image, tf.float32) / 255.0

  # Load a pre-trained CNN model (e.g. MobileNetV2) and get the output of the last hidden layer
  cnn = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False)
  cnn.trainable = False
  features = cnn(image[tf.newaxis, ...])

  # Return the features as a tensor
  return features

# Define a function to generate a caption for the image using an image-to-text model (e.g. Show and Tell)
def image_caption(image):
  # Load a pre-trained image-to-text model and its vocabulary
  caption_model = tf.saved_model.load('show_and_tell')
  vocab = tf.saved_model.load('vocab')

  # Preprocess the image and get its features
  features = image_preprocess(image)

  # Generate a caption using beam search
  caption = caption_model.generate(features, vocab, beam_size=5)

  # Return the caption as a string
  return caption

# Define a function to evaluate the essay based on the image and the BERT model
def essay_evaluate(image, essay):
  # Generate a caption for the image using the image-to-text model
  caption = image_caption(image)

  # Tokenize the essay and the caption using the BERT tokenizer
  input_ids = tokenizer([essay, caption])

  # Get the logits from the BERT model
  logits = model(input_ids)['logits']

  # Get the probabilities of each class (0: bad, 1: good) using softmax
  probs = tf.nn.softmax(logits)

  # Get the predicted class and its probability for the essay
  pred_class = tf.argmax(probs[0]).numpy()
  pred_prob = probs[0][pred_class].numpy()

  # Return the predicted class and its probability as a tuple
  return (pred_class, pred_prob)


In [2]:
!pip install imagen


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for imagen: filename=imagen-2.1.0-py3-none-any.whl size=65218 sha256=336428b3f614ba77ae3feafaa59894630f28df5ebb7830170860801945a53e91
  Stored in directory: /root/.cache/pip/wheels/5b/02/06/9b225d63cfe4a82bbdd4494a0a415b7c615f74a80057b94dbc
Successfully built imagen


In [4]:
def image_preprocess(image):
  # Resize and normalize the image
  image = tf.image.resize(image, (224, 224))
  image = tf.cast(image, tf.float32) / 255.0

  # Load a pre-trained CNN model (e.g. MobileNetV2) and get the output of the last hidden layer
  cnn = tf.keras.applications.MobileNetV2(weights='imagenet', include_top=False)
  cnn.trainable = False
  features = cnn(image[tf.newaxis, ...])

  # Return the features as a tensor
  return features

In [10]:
!pip install transformers




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.0 MB/s eta 0:00:00


In [11]:

from transformers import pipeline

image_to_text = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")

image_to_text("/content/gestrue06_outside.jpeg")

# [{'generated_text': 'a soccer game with a player jumping to catch the ball '}]


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1346: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'a man with a beard is talking on a cell phone '}]

In [15]:

from transformers import VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
import torch
from PIL import Image

model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
tokenizer = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



max_length = 160
num_beams = 20
gen_kwargs = {"max_length": max_length, "num_beams": num_beams}
def predict_step(image_paths):
  images = []
  for image_path in image_paths:
    i_image = Image.open(image_path)
    if i_image.mode != "RGB":
      i_image = i_image.convert(mode="RGB")

    images.append(i_image)

  pixel_values = feature_extractor(images=images, return_tensors="pt").pixel_values
  pixel_values = pixel_values.to(device)

  output_ids = model.generate(pixel_values, **gen_kwargs)

  preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
  preds = [pred.strip() for pred in preds]
  return preds


predict_step(['/content/gestrue06_outside.jpeg']) # ['a woman in a hospital bed with a woman in a hospital bed']


['a man with a beard talking on a cell phone']

OSError: ignored